<a href="https://colab.research.google.com/github/kaushal-k-k/CS6910/blob/main/Assignment3/A3Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xvf  'dakshina_dataset_v1.0.tar'

--2021-05-10 13:52:22--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 172.217.13.80, 172.217.13.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  27.3MB/s    in 21s     

2021-05-10 13:52:43 (91.8 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
d

# New Section

In [ ]:
%pip install wandb -q
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8MB 19.1MB/s 
     |████████████████████████████████| 133kB 46.4MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 163kB 53.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 


In [ ]:
input_token_index = None
target_token_index = None
MAX_LEN_input = None
MAX_LEN_target = None
num_encoder_tokens = 70
num_decoder_tokens = 30
input_tokenizer = None

In [ ]:
def tokenize(data,vocab_size):
  tokenizer = Tokenizer(num_words=vocab_size,char_level=True)
  tokenizer.fit_on_texts(data)
  temp=tokenizer.texts_to_sequences(data)
  # print(data[:3])
  # print(temp[:3])
  dictionary = tokenizer.word_index
  return temp , dictionary , tokenizer

In [ ]:
def load_and_preprocess():
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    input_text, target_text = temp[0],temp[1] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  MAX_LEN_input = max([len(txt) for txt in input_texts])
  MAX_LEN_target = max([len(txt) for txt in target_texts])

  # toeknize
  encoder_input , input_token_index , input_tokenizer = tokenize(input_texts , num_encoder_tokens)
  decoder_input , target_token_index,_ = tokenize(target_texts , num_decoder_tokens) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data



In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Input,GRU,SimpleRNN,Dropout,Embedding

In [ ]:
def create_model(m_name="LSTM",n_e_layers=1,n_d_layers=1,latent_dim = 100,embedding_size = 16,dropout = 0 , recurrent_dropout = 0):
  keras.backend.clear_session()
  # num_encoder_tokens = 30
  # num_decoder_tokens = 50
  # Define an input sequence and process it.
  input1 = Input(shape=(None,),name= "input_1")
  encoder_inputs = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(input1)

  encoder = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True,return_sequences=True)
  e_o = encoder(encoder_inputs)
  prev = e_o
  for i in range(1,n_e_layers):
    e = globals()[m_name](latent_dim, dropout=dropout,recurrent_dropout = recurrent_dropout,return_state=True,return_sequences=True)
    e_o = e(prev[0])
    prev = e_o
  
  input2 = Input(shape=(None,),name="input_2")
  decoder_inputs = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(input2)
  d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_sequences=True, return_state=True)(decoder_inputs,initial_state = e_o[1:])
  p_d = d_l[0]
  for i in range(1,n_d_layers):
    d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True, return_sequences=True)(p_d,initial_state = e_o[1:])
    p_d = d_l[0]

  
  decoder_dense = Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(d_l[0])

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([input1,input2], decoder_outputs)

  return model


In [ ]:
def train():
  run = wandb.init()
  c = run.config
  name = "model_"+c.model+"_el_"+str(c.encoder_layers)+"_dl_"+str(c.decoder_layers)+"_hs_"+str(c.hidden_size)+"_em_"+str(c.embedding_size)+"_d_"+str(c.dropout)
  run.name = name
  print(name)
  batch_size = 64
  epochs = 10

  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model = create_model(c.model,c.encoder_layers,c.decoder_layers,c.hidden_size,c.embedding_size,c.dropout,0)
  model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[WandbCallback(),es]
  )
  return

In [ ]:
sweep_config={
    'method' : 'random' ,
    'metric' : { 'name' : 'val_accuracy' , 'goal' : 'maximize' } ,
    'parameters' : {
        'model' : { 'values' : ['LSTM','GRU','SimpleRNN'] },
        'dropout' : { 'values' : [0.1,0.2]},
        'embedding_size' : {'values' : [16,32,64]},
        'encoder_layers' : {'values' : [1,2,3]},
        'decoder_layers' : {'values' : [1,2,3]},
        'hidden_size' : {'values' : [64,128,256]}
    }
}


In [ ]:

sweepid = wandb.sweep(sweep_config,project="DL_A3_Q1",entity ="sonagara")
wandb.agent(sweepid,train)

Create sweep with ID: zwft5zyz
Sweep URL: https://wandb.ai/sonagara/dl_a3_q1/sweeps/zwft5zyz


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4ibogenx with config:
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.1
wandb: 	embedding_size: 16
wandb: 	encoder_layers: 3
wandb: 	hidden_size: 64
wandb: 	model: LSTM


model_LSTM_el_3_dl_1_hs_64_em_16_d_0.1
1316/1316 [==============================] - 27s 16ms/step - loss: 1.0530 - accuracy: 0.7261 - val_loss: 0.9644 - val_accuracy: 0.7347


epoch,0
loss,0.89388
accuracy,0.7442
val_loss,0.96437
val_accuracy,0.73466
_runtime,34
_timestamp,1620655704
_step,0
best_val_loss,0.96437
best_epoch,0


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


...

epoch,0
loss,0.88402
accuracy,0.74622
val_loss,0.96089
val_accuracy,0.74472
_runtime,41
_timestamp,1620655827
_step,0
best_val_loss,0.96089
best_epoch,0


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: ytlyw6m9 with config:
wandb: 	decoder_layers: 3
wandb: 	dropout: 0.2
wandb: 	embedding_size: 16
wandb: 	encoder_layers: 1
wandb: 	hidden_size: 256
wandb: 	model: GRU


model_GRU_el_1_dl_3_hs_256_em_16_d_0.2
 322/1316 [======>.......................] - ETA: 15s - loss: 1.2263 - accuracy: 0.6959

wandb: Ctrl + C detected. Stopping sweep.
